In [6]:
import pandas as pd
from xml.dom import minidom

In [2]:
def create_element(text):
    return dom.createElement(text)

In [3]:
def create_text(text):
    return dom.createTextNode(text)

In [4]:
dom = minidom.Document()
root_node = create_element('zabbix_export')
dom.appendChild(root_node)

<DOM Element: zabbix_export at 0x7fa4e64e34b0>

In [8]:
xlsx_file = './Test//host.xlsx'
data = pd.read_excel(xlsx_file)

In [6]:
root_node.appendChild(create_element('version')).appendChild(create_text('5.0'))
root_node.appendChild(create_element('groups')).appendChild(create_element('group')).appendChild(create_element('name')).appendChild(create_text('IPMI'))
hosts = root_node.appendChild(create_element('hosts'))
for i in range(data.shape[0]):
    host = hosts.appendChild(create_element('host'))
    host.appendChild(create_element('host')).appendChild(create_text(data.loc[i]['host']))
    host.appendChild(create_element('name')).appendChild(create_text(data.loc[i]['name']))
    host.appendChild(create_element('ipmi_username')).appendChild(create_text(data.loc[i]['username']))
    host.appendChild(create_element('ipmi_password')).appendChild(create_text(data.loc[i]['password']))
    host.appendChild(create_element('groups')).appendChild(create_element('group')).appendChild(create_element('name')).appendChild(create_text('IPMI'))
    interface = host.appendChild(create_element('interfaces')).appendChild(create_element('interface'))
    interface.appendChild(create_element('type')).appendChild(create_text('IPMI'))
    interface.appendChild(create_element('ip')).appendChild(create_text(data.loc[i]['host']))
    interface.appendChild(create_element('port')).appendChild(create_text('623'))
    interface.appendChild(create_element('interface_ref')).appendChild(create_text('if1'))
    host.appendChild(create_element('inventory_mode')).appendChild(create_text('DISABLED'))

In [7]:
try:
    with open('./host_ipmi.xml','w',encoding='UTF-8') as fh:
        dom.writexml(fh,indent='',addindent='\t',newl='\n',encoding='UTF-8')
        print('写入xml OK!')
except Exception as err:
    print('错误信息：{0}'.format(err))

写入xml OK!


In [2]:
import os
from pyzabbix import ZabbixAPI
import json
zapi = ZabbixAPI('http://192.168.50.100:18080')
zapi.login('Admin','zabbix')

In [10]:
rules = {
    'applications': {
        'createMissing': True,
    },
    'discoveryRules': {
        'createMissing': True,
        'updateExisting': True
    },
    'graphs': {
        'createMissing': True,
        'updateExisting': True
    },
    'groups': {
        'createMissing': True
    },
    'hosts': {
        'createMissing': True,
        'updateExisting': True
    },
    'images': {
        'createMissing': True,
        'updateExisting': True
    },
    'items': {
        'createMissing': True,
        'updateExisting': True
    },
    'maps': {
        'createMissing': True,
        'updateExisting': True
    },
    'screens': {
        'createMissing': True,
        'updateExisting': True
    },
    'templateLinkage': {
        'createMissing': True,
    },
    'templates': {
        'createMissing': True,
        'updateExisting': True
    },
    'templateScreens': {
        'createMissing': True,
        'updateExisting': True
    },
    'triggers': {
        'createMissing': True,
        'updateExisting': True
    },
    'valueMaps': {
        'createMissing': True,
        'updateExisting': True
    },
}
file = './host_ipmi.xml'
if os.path.isfile(file):
    #file = path/*.xml
    print(file)
    with open(file, 'r') as f:
        template = f.read()
        try:
            zapi.confimport('xml', template, rules)
        except ZabbixAPIException as e:
            print(e)
    print('')
else:
    print('I need a xml file')

./host_ipmi.xml



### 获取hostid

In [36]:
host_list = zapi.host.get(
    output=['hostid', 'host'],
)

In [37]:
host_list

[{'hostid': '10355', 'host': '192.168.50.25'},
 {'hostid': '10357', 'host': '192.168.50.71'},
 {'hostid': '10359', 'host': '192.168.50.72'},
 {'hostid': '10360', 'host': '192.168.50.73'},
 {'hostid': '10363', 'host': '192.168.50.251'},
 {'hostid': '10364', 'host': '192.168.50.1'},
 {'hostid': '10369', 'host': '192.168.50.190'},
 {'hostid': '10373', 'host': '192.168.50.53'},
 {'hostid': '10383', 'host': '192.168.50.11'},
 {'hostid': '10385', 'host': '192.168.50.112'},
 {'hostid': '10388', 'host': '192.168.50.23'},
 {'hostid': '10403', 'host': '192.168.50.20'},
 {'hostid': '10404', 'host': '192.168.50.22'}]

In [14]:
data

,host,name,username,password
0,192.168.50.20,IPMI: 192.168.50.111,root,Dacheng@123
1,192.168.50.22,IPMI: 192.168.50.112,root,Dacheng@123


In [33]:
host_dict = {}
for i in host_list:
    if i['host'] in list(data['host']):
        host_dict[i['hostid']] = i['host']
host_dict

{'10403': '192.168.50.20', '10404': '192.168.50.22'}

### 获取templateid

In [17]:
template_list = zapi.template.get(
    output=["templateid","name"],
)

In [18]:
for i in template_list:
    if i['name'] == 'Template ipmi_get of IPMI':
        print(i['templateid'])

10401


In [25]:
def get_template_id(template_list):
    '''获取所有模板的id和名字信息'''
    template_list = zapi.template.get(output=["templateid","name"])

    return int([i['templateid'] for i in template_list if i['name'] == 'Template ipmi_get of IPMI'][0])

In [26]:
get_template_id(template_list)

10402

### 新增itemid

In [8]:
hosts = zapi.host.get(filter={"host": '192.168.50.20'}, selectInterfaces=["interfaceid"])

In [35]:
for i in host_dict:
    print(i,host_dict[i])

10403 192.168.50.20
10404 192.168.50.22


In [19]:
add_template = zapi.template.massadd(templates=10401, hosts={"hostid": 10399})

In [31]:
hosts[0]["interfaces"][0]["interfaceid"]

'28'

### 获取数据

In [27]:
import sched,time
def func(a):
  print(time.time(),"Hello Sched!",a)
print(time.time())
s = sched.scheduler(time.time,time.sleep)
# 2为延后时间，1为优先级，func为函数名，("test1",)为函数参数
s.enter(60,0,func,("1分钟",))
s.run()
print(time.time())

1619415847.5371385
1619415907.5827217 Hello Sched! 1分钟
1619415907.5829704


In [32]:
item_list = zapi.item.get(
    hostids='10399',
    search = {
        "key_" : 'ipmi.get'
    },
    output="extend",
)

### 删除模板

In [37]:
del_template = zapi.host.update(hostid=10399,templates_clear={"templateid": 10401})